---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [3]:
count = 1
graph_dicts = {}
for graph in P1_Graphs:
    degree_unique_vals = sorted(set(graph.degree().values()), reverse=True)
    graph_dicts['graph-'+str(count)+'-degree-distribution'] = [(i, list(graph.degree().values()).count(i)/float(graph.number_of_nodes())) for i in degree_unique_vals]
    graph_dicts['graph-'+str(count)+'-clustering'] = nx.average_clustering(graph)
    graph_dicts['graph-'+str(count)+'-node_count'] = graph.number_of_nodes()
    graph_dicts['graph-'+str(count)+'-shortest-path'] = nx.average_shortest_path_length(graph)
    count=count+1

In [38]:
# degree distribution analysis
graph_dicts['graph-1-degree-distribution']

[(100, 0.001),
 (49, 0.001),
 (39, 0.002),
 (38, 0.001),
 (30, 0.002),
 (29, 0.001),
 (27, 0.002),
 (26, 0.002),
 (22, 0.002),
 (21, 0.001),
 (20, 0.002),
 (19, 0.003),
 (18, 0.002),
 (17, 0.002),
 (16, 0.001),
 (15, 0.001),
 (14, 0.002),
 (13, 0.005),
 (12, 0.005),
 (11, 0.007),
 (10, 0.009),
 (9, 0.014),
 (8, 0.016),
 (7, 0.026),
 (6, 0.032),
 (5, 0.056),
 (4, 0.1),
 (3, 0.237),
 (2, 0.465)]

In [5]:
# # clustering distribution analysis
for i in range(1,6):
    print(graph_dicts['graph-{}-clustering'.format(i)])

0.03167539146454044
0.5642419635919628
0.4018222222222227
0.03780379975223251
0.0033037037037037037


In [6]:
# # shortest path distribution analysis
for i in range(1,6):
    print(graph_dicts['graph-{}-shortest-path'.format(i)])

4.099161161161161
5.089871871871872
9.378702269692925
3.1048046283934134
5.0785509568313305


In [7]:
# # node count in each graph
for i in range(1,6):
    print(graph_dicts['graph-{}-node_count'.format(i)])

1000
1000
750
750
750


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [8]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_H','SW_L','PA','SW_H']

In [9]:
graph_identification()

['PA', 'SW_H', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [10]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [11]:
# nodes = G.nodes(data=True)
# df = pd.DataFrame(index = G.nodes())
# df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
# df['management-salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
# df['degrees'] = pd.Series(nx.degree(G))
# df['clustering'] = pd.Series(nx.clustering(G))
# # node_attributes_from_graph = [(node[0], node[1]['Department'], node[1]['ManagementSalary']) for node in nodes]

In [12]:
def mgmt_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    mgmt_emails = 0
    for edge in edges:
        if salary_type_dict[edge[1]] == 1:
            mgmt_emails=mgmt_emails+1
    row['mgmt_communication'] = mgmt_emails
    return row



In [13]:
def non_mgmt_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    non_mgmt_emails = 0
    for edge in edges:
        if salary_type_dict[edge[1]] == 0:
            non_mgmt_emails=non_mgmt_emails+1
    row['non_mgmt_communication'] = non_mgmt_emails
    return row

In [14]:
def unknown_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    unknown_emails = 0
    for edge in edges:
        if np.isnan(salary_type_dict[edge[1]]):
            unknown_emails=unknown_emails+1
    row['unknown_communication'] = unknown_emails
    return row

In [15]:
# df = (df.reset_index()
#         .rename(columns={'index': 'nodes'})
#         .apply(mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
#         .apply(non_mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
# #         .apply(unknown_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
#      )

In [16]:
# df_training = df[df['management-salary'].isnull() == False]
# df_training.info()

In [17]:
# df_prediction = df[df['management-salary'].isnull()]
# df_prediction.info()

In [18]:
# logistic_model.score(X_test,y_test)

In [19]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df_training.loc[:, df_training.columns !='management-salary'],df_training.loc[:, 'management-salary'], 
#                                                     train_size= 0.8, random_state = 0)

In [20]:
# from sklearn.linear_model import LogisticRegression
# logistic_model = LogisticRegression(C=1000)
# logistic_model.fit(X_train,y_train)
# logistic_model.score(X_train,y_train)

# prediction = logistic_model.predict(df_prediction.loc[:, df_training.columns !='management-salary'])
# prediction_probas = logistic_model.predict_proba(df_prediction.loc[:, df_training.columns !='management-salary'])
# proba_mgmt_salary = [ proba[1] for proba in prediction_probas]
# mgmt_salary_proba_series = pd.Series(proba_mgmt_salary, index = df_prediction.index)

In [21]:
def salary_predictions():
    nodes = G.nodes(data=True)
    df = pd.DataFrame(index = G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['management-salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['degrees'] = pd.Series(nx.degree(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    # node_attributes_from_graph = [(node[0], node[1]['Department'], node[1]['ManagementSalary']) for node in nodes]

    df = (df.reset_index()
            .rename(columns={'index': 'nodes'})
            .apply(mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
            .apply(non_mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
    #         .apply(unknown_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
         )
    
    df_training = df[df['management-salary'].isnull() == False]
    df_prediction = df[df['management-salary'].isnull()]
    
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(df_training.loc[:, df_training.columns !='management-salary'],df_training.loc[:, 'management-salary'], 
                                                    train_size= 0.8, random_state = 0)
    
    from sklearn.linear_model import LogisticRegression
    logistic_model = LogisticRegression(C=1000)
    logistic_model.fit(X_train,y_train)
    logistic_model.score(X_train,y_train)
    
    prediction = logistic_model.predict(df_prediction.loc[:, df_training.columns !='management-salary'])
    
    prediction_probas = logistic_model.predict_proba(df_prediction.loc[:, df_training.columns !='management-salary'])
    proba_mgmt_salary = [ proba[1] for proba in prediction_probas]
    mgmt_salary_proba_series = pd.Series(proba_mgmt_salary, index = df_prediction.index)
    # Your Code Here
    
    return mgmt_salary_proba_series

In [22]:
salary_predictions()

1       0.075913
2       0.734868
5       0.957095
8       0.073657
14      0.149857
18      0.115260
27      0.257596
30      0.415210
31      0.220136
34      0.153350
37      0.078485
40      0.177223
45      0.110995
54      0.370649
55      0.326768
60      0.169890
62      0.997971
65      0.208667
77      0.061647
79      0.108062
97      0.057857
101     0.030756
103     0.170964
108     0.054757
113     0.569090
122     0.034083
141     0.235956
142     0.932950
144     0.094260
145     0.580716
          ...   
913     0.026959
914     0.081433
915     0.002755
918     0.123238
923     0.044536
926     0.086469
931     0.050468
934     0.015617
939     0.010486
944     0.003463
945     0.048302
947     0.187823
950     0.063947
951     0.044451
953     0.018898
959     0.002736
962     0.002684
963     0.140773
968     0.155656
969     0.140170
974     0.080190
984     0.016932
987     0.101720
989     0.146349
991     0.192289
992     0.003161
994     0.003438
996     0.0031

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [23]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [24]:
future_connections.info()

<class 'pandas.core.frame.DataFrame'>
Index: 488446 entries, (6, 840) to (75, 101)
Data columns (total 1 columns):
Future Connection    366334 non-null float64
dtypes: float64(1)
memory usage: 7.5+ MB


In [25]:
# comm_net_score = list(nx.cn_soundarajan_hopcroft(G, community='ManagementSalary'))

In [26]:
# cn_dict = {}
# for cn_score in comm_net_score:
#     cn_dict[(cn_score[0], cn_score[1])] = cn_score[2]

In [27]:
# jaccard_coeff = list(nx.jaccard_coefficient(G))

In [28]:
# jacc_dict = {}
# for jacc_score in jaccard_coeff:
#     jacc_dict[(jacc_score[0], jacc_score[1])] = jacc_score[2]

In [29]:
# future_connections = (future_connections.reset_index()
#                                         .rename(columns={'index' : 'edge'})
#                      )

In [30]:
# future_connections.head()

In [31]:
# future_connections['cn-score'] = future_connections['edge'].apply(lambda x : cn_dict[x])

In [32]:
# future_connections.set_index('edge', drop=True, inplace=True)

In [33]:
# future_connections.head()

In [34]:
# from sklearn.model_selection import train_test_split

# future_connections_train = future_connections[future_connections['Future Connection'].isnull() == False]
# future_connections_predict = future_connections[future_connections['Future Connection'].isnull()]

In [35]:


# X_train, X_test, y_train, y_test = train_test_split(future_connections_train['cn-score'], future_connections_train['Future Connection'], 
#                                                 train_size= 0.8)

# from sklearn.linear_model import LogisticRegression
# logistic_model = LogisticRegression(C=100)
# logistic_model.fit(X_train.reshape(X_train.shape[0],1), y_train.reshape(y_train.shape[0],1))
# logistic_model.score(X_test.reshape(X_test.shape[0],1), y_test.reshape(y_test.shape[0],1))

# # prediction = logistic_model.predict(future_connections_predict.loc[:, df_training.columns !='management-salary'])

# prediction_probas = logistic_model.predict_proba(future_connections_predict['cn-score'].reshape(future_connections_predict['cn-score'].shape[0],1))
# proba_future_connect = [ proba[1] for proba in prediction_probas]
# future_connect_series = pd.Series(proba_mgmt_salary, index = future_connections_predict.index)

In [36]:
def new_connections_predictions():
    comm_net_score = list(nx.cn_soundarajan_hopcroft(G, community='ManagementSalary'))

    cn_dict = {}
    for cn_score in comm_net_score:
        cn_dict[(cn_score[0], cn_score[1])] = cn_score[2]

    future_connection = (future_connections.reset_index()
                                            .rename(columns={'index' : 'edge'})
                         )

    future_connection['cn-score'] = future_connection['edge'].apply(lambda x : cn_dict[x])

    future_connection.set_index('edge', drop=True, inplace=True)

    from sklearn.model_selection import train_test_split

    future_connections_train = future_connection[future_connection['Future Connection'].isnull() == False]
    future_connections_predict = future_connection[future_connection['Future Connection'].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(future_connections_train['cn-score'], future_connections_train['Future Connection'], 
                                                    train_size= 0.8)

    from sklearn.linear_model import LogisticRegression
    logistic_model = LogisticRegression(C=100)
    logistic_model.fit(X_train.reshape(X_train.shape[0],1), y_train.reshape(y_train.shape[0],1))
    logistic_model.score(X_test.reshape(X_test.shape[0],1), y_test.reshape(y_test.shape[0],1))

    # prediction = logistic_model.predict(future_connections_predict.loc[:, df_training.columns !='management-salary'])

    prediction_probas = logistic_model.predict_proba(future_connections_predict['cn-score'].reshape(future_connections_predict['cn-score'].shape[0],1))
    proba_future_connect = [ proba[1] for proba in prediction_probas]
    future_connect_series = pd.Series(proba_future_connect, index = future_connections_predict.index)
    
    return future_connect_series

In [37]:
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


edge
(107, 348)    0.033871
(542, 751)    0.013903
(20, 426)     0.678855
(50, 989)     0.013903
(942, 986)    0.013903
(324, 857)    0.013903
(13, 710)     0.178164
(19, 271)     0.178164
(319, 878)    0.013903
(659, 707)    0.013903
(49, 843)     0.013903
(208, 893)    0.013903
(377, 469)    0.013903
(405, 999)    0.021749
(129, 740)    0.021749
(292, 618)    0.021749
(239, 689)    0.013903
(359, 373)    0.013903
(53, 523)     0.033871
(276, 984)    0.013903
(202, 997)    0.013903
(604, 619)    0.033871
(270, 911)    0.013903
(261, 481)    0.080189
(200, 450)    0.678855
(213, 634)    0.013903
(644, 735)    0.052389
(346, 553)    0.013903
(521, 738)    0.013903
(422, 953)    0.021749
                ...   
(672, 848)    0.013903
(28, 127)     0.892346
(202, 661)    0.013903
(54, 195)     0.998003
(295, 864)    0.013903
(814, 936)    0.013903
(839, 874)    0.013903
(139, 843)    0.013903
(461, 544)    0.013903
(68, 487)     0.013903
(622, 932)    0.013903
(504, 936)    0.021749
(479, 